NOTE: The github viewer doesnt support custom css in a svg at this time. To view this notebook correctly, use
an external viewer something like https://jsvine.github.io/nbpreview/.

First lets import poloto.

In [70]:
:dep poloto

Next import evcxr rendering for poloto.

In [71]:
:dep poloto-evcxr
use poloto_evcxr::prelude::*;

Lets load just one css for all poloto graphs in this notebook.

In [72]:
let s=r##"
.poloto{
    stroke-linecap:round;
    stroke-linejoin:round;
    font-family:Roboto,sans-serif;
    font-size:16px;
  }
  .poloto_background{fill:AliceBlue;}
  .poloto_scatter{stroke-width:7}
  .poloto_line{stroke-width:2}
  .poloto_text{fill: black;}
  .poloto_name{font-size:24px;dominant-baseline:auto;text-anchor:middle;}
  .poloto_where{dominant-baseline:middle;text-anchor:start}
  .poloto_text.poloto_legend{font-size:20px;dominant-baseline:middle;text-anchor:start;}
  .poloto_text.poloto_ticks.poloto_y{dominant-baseline:middle;text-anchor:end}
  .poloto_text.poloto_ticks.poloto_x{dominant-baseline:auto;text-anchor:middle}
  .poloto_imgs.poloto_ticks{stroke: black;stroke-width:3;fill:none;stroke-dasharray:none}
  .poloto_grid{stroke:gray;stroke-width:0.5}
  
  .poloto0.poloto_stroke{stroke:blue;}
  .poloto1.poloto_stroke{stroke:red;}
  .poloto2.poloto_stroke{stroke:green;}
  .poloto3.poloto_stroke{stroke:gold;}
  .poloto4.poloto_stroke{stroke:aqua;}
  .poloto5.poloto_stroke{stroke:lime;}
  .poloto6.poloto_stroke{stroke:orange;}
  .poloto7.poloto_stroke{stroke:chocolate;}
  .poloto0.poloto_fill{fill:blue;}
  .poloto1.poloto_fill{fill:red;}
  .poloto2.poloto_fill{fill:green;}
  .poloto3.poloto_fill{fill:gold;}
  .poloto4.poloto_fill{fill:aqua;}
  .poloto5.poloto_fill{fill:lime;}
  .poloto6.poloto_fill{fill:orange;}
  .poloto7.poloto_fill{fill:chocolate;}
"##;


let s=format!("<style>{}</svg>",s);

println!("EVCXR_BEGIN_CONTENT text/html\n{}\nEVCXR_END_CONTENT", s);

Now lets make a simple function that just displays one plot to avoid having to write alot of boilerplate.

In [73]:
// import hypermelon crate so that we can manually append elements.
:dep hypermelon
fn show(it:impl IntoIterator<Item=[f64;2]>){
    
    use hypermelon::prelude::*;
    let j=poloto::build::plot("a").line(it.into_iter());
    let svg = poloto::header().with_dim([400.0,300.0]);
    let style = poloto::render::Theme::light();
    let res=poloto::frame()
        .with_tick_lines([true, true])
        .build()
        .data(j)
        .build_and_label(("hello","x","y"));
    res.append_to(svg).render_evcxr();
}

Ok now lets plot some data.

In [74]:
let data=vec!([0.0,0.0],[1.0,1.0],[3.0,5.0]);
show(data);

a hello x y 0 1 2 3 4 5 0.0 0.5 1.0 1.5 2.0 2.5 3.0

Ok cool. Now lets change the data.

In [75]:
let data=vec!([1.0,5.0],[2.0,1.0],[3.0,0.0]);
show(data);

a hello x y 0 1 2 3 4 5 1.0 1.5 2.0 2.5 3.0

Oh no now the data is going down!

Here is an gaussian example

In [76]:
{
    use poloto::build;

    // See https://en.wikipedia.org/wiki/Gaussian_function
    let gau = |sigma: f64, mu: f64| {
        use std::f64::consts::TAU;
        let s = sigma.powi(2);
        let k = (sigma * TAU).sqrt().recip();
        move |x: f64| [x, (-0.5 * (x - mu).powi(2) / s).exp() * k]
    };

    let xs = poloto::util::range_iter([-5.0, 5.0], 200);

    let plots = poloto::plots!(
        build::plot("σ=1.0").line(xs.clone().map(gau(1.0, 0.0))),
        build::plot("σ=0.5").line(xs.clone().map(gau(0.5, 0.0))),
        build::plot("σ=0.3").line(xs.clone().map(gau(0.3, 0.0)))
    );

    poloto::frame_build()
        .data(poloto::plots!(build::origin(), plots))
        .build_and_label(("gaussian", "x", "y"))
        .append_to(poloto::header())
        .render_evcxr();
}

()

Here is a collatz example

In [77]:
{
    use hypermelon::prelude::*;
    use poloto::build;

    let collatz = |mut a: i128| {
    std::iter::from_fn(move || {
            if a == 1 {
                None
            } else {
                a = if a % 2 == 0 { a / 2 } else { 3 * a + 1 };
                Some(a)
            }
        })
        .fuse()
    };

    let svg = poloto::header().with_viewbox_width(1200.0);

    //let style =
    //    poloto::render::Theme::dark().append(".poloto_line{stroke-dasharray:2;stroke-width:2;}");

    let a = (1000..1006).map(|i| build::plot(format!("c({})", i)).line((0..).zip(collatz(i))));

    poloto::frame()
        .with_tick_lines([true, true])
        .with_viewbox(svg.get_viewbox())
        .build()
        .data(poloto::plots!(poloto::build::origin(), a))
        .build_and_label(("collatz", "x", "y"))
        .append_to(svg)
        .render_evcxr();
}

()